# MNIST uDNN Model Pre-training + Capuchin
Credit: this notebook is based on Keras official documents https://keras.io/examples/vision/mnist_convnet/

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1-Bn1Z7XItLokDJanKX9I3KlBWWjXI-RE?usp=sharing)

In [ ]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
import matplotlib.pyplot as plt

## Use Google Colab Pro GPU (Optional)

In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

## Import MNIST Dataset

In [ ]:
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# the data, split between train and test sets
(trainX, trainY), (testX, testY) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
trainX = trainX.astype("float32") / 255
testX = testX.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
trainX = np.expand_dims(trainX, -1)
testX = np.expand_dims(testX, -1)
print("trainX shape:", trainX.shape)
print(trainX.shape[0], "train samples")
print(testX.shape[0], "test samples")


# convert class vectors to binary class matrices
trainY = keras.utils.to_categorical(trainY, num_classes)
testY = keras.utils.to_categorical(testY, num_classes)

## Define uDNN Model

In [ ]:
model = keras.Sequential([
        keras.Input(shape=input_shape),
        layers.Conv2D(16, kernel_size=(3,3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2,2)),
        layers.Conv2D(32, kernel_size=(3,3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2,2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
])

model.summary()

## Compile and Train Model

In [ ]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
model.fit(trainX, trainY, epochs=15, batch_size=128, validation_split=0.1)

In [ ]:
model.evaluate(testX, testY)

## Prepare to Export Model

In [ ]:
! git clone https://github.com/leleonardzhang/Capuchin.git
! pip install fxpmath
import sys
from fxpmath import Fxp
sys.path.insert(0, '/content/Capuchin')
import encoder

## Export Model to Header File

In [ ]:
encoder.export_model(model)

## Get Sample Input and Output (Optional)

In [ ]:
test_n = 1      # the nth data in test dataset
print("Fixed Point Input")
print(Fxp(testX[test_n].transpose(2,0,1), signed = True, n_word = 16, n_frac = 10).val.flatten().tolist())
print("Output Label")
print(np.argmax(testY[test_n]))

## Next Step

1.   Download header file `neural_network_parameters.h` and copy the header file into the root directory of MSP implementation `Capuchin-MCU/`.
2.   Modify the pointer `input_buffer` in `neural_network_parameters.h` and point to desired data input.
3.   Compile and run the program on MSP.
